In [23]:
from pytrends.request import TrendReq
from pytrends.exceptions import ResponseError
import time

In [24]:
def fetch_trend_data(keywords, timeframe="today 12-m", geo="", gprop="", max_retries=5):
    """
    Fetch Google Trends data for given keyword(s), with retry on HTTP 429.

    Args:
        keywords (str or list of str): Single term or list of terms to query.
        timeframe (str): Time period (e.g. "2023-01-01 2023-06-01" or "today 12-m").
        geo (str): Country code (e.g. "US", "IL"); empty string for worldwide.
        gprop (str): Category (e.g. "news", "images", "youtube"); empty for web search.
        max_retries (int): Number of attempts if a 429 (rate-limit) is encountered.

    Returns:
        pandas.DataFrame or None: Interest over time, or None on failure.
    """
    # Ensure keywords is a non-empty list of strings
    kw_list = [keywords] if isinstance(keywords, str) else keywords
    if not kw_list or any(not isinstance(k, str) or not k.strip() for k in kw_list):
        raise ValueError("keywords must be a non-empty string or list of non-empty strings")

    pytrend = TrendReq(hl="en-US", tz=0)
    attempt = 0
    backoff = 1  # initial backoff in seconds

    while attempt < max_retries:
        try:
            pytrend.build_payload(kw_list=kw_list, timeframe=timeframe, geo=geo, gprop=gprop)
            df = pytrend.interest_over_time()
            if df.empty:
                print(f"No data returned for {kw_list} over '{timeframe}'.")
                return None
            return df

        except ResponseError as e:
            msg = str(e).lower()
            # If it's a 429 (rate-limit), wait and retry
            if "429" in msg:
                attempt += 1
                print(f"Rate limited (429). Retry {attempt}/{max_retries} after {backoff}s...")
                time.sleep(backoff)
                backoff *= 2  # exponential backoff
                continue
            else:
                # Other errors: print and abort
                print(f"ResponseError: {e}. Please check parameters.")
                return None

    print(f"Failed to fetch after {max_retries} retries due to repeated 429 errors.")
    return None


In [25]:
df = fetch_trend_data("Python", timeframe="2024-01-01 2024-06-01", geo="US", gprop="shopping")


ValueError: gprop must be empty (to indicate web), images, news, youtube, or froogle

In [ ]:
df.drop(columns=['isPartial'], errors='ignore').drop(columns=['isPartial'], errors='ignore')

,Python
date,
2024-01-01,24
2024-01-02,40
2024-01-03,51
2024-01-04,50
2024-01-05,47
...,...
2024-05-28,49
2024-05-29,60
2024-05-30,57


In [ ]:
import pandas as pd
import datetime
datetime.del('12-m')

TypeError: 'str' object cannot be interpreted as an integer